## Day 9

https://adventofcode.com/2022/day/9#part2

In [1]:
def parse09(filename):
    with open(filename) as f:
        data = [ [ l.strip().split(" ")[0], int(l.strip().split(" ")[1]) ] for l in f.readlines() ]
        return data

In [2]:
from collections import defaultdict

M = {"R": (+1,0), "L":(-1,0), "U":(0,+1), "D":(0,-1)}

def moveHead(c,m):
    return tuple([ i+j for i,j in zip(c,M[m]) ])

def moveTail(H,Hprev,T):
    # same position, don't move
    if H==T: 
        return T
    xH,yH = H
    xT,yT = T
    dx = abs(xH-xT)
    dy = abs(yH-yT)
    if dx<=1 and dy<=1: # adiacent, don't move
        return T
    else:
        if dx<=1 and dy==2:
            return (xH,yH-(yH-yT)//dy)
        elif dx==2 and dy<=1:
            return (xH-(xH-xT)//dx,yH)
        elif dx==2 and dy==2:
            return (xH-(xH-xT)//dx,yH-(yH-yT)//dy)

def part1(instr):
    H = (0,0)
    T = (0,0)
    Ht = defaultdict(int)
    Tt = defaultdict(int)
    Ht[H] = 1
    Tt[H] = 1
    for d,n in instr:
        for _ in range(n):
            # move Head
            Hprev = H
            H = moveHead(Hprev,d)
            Ht[H] += 1
            # move Tail
            T = moveTail(H,Hprev,T)
            Tt[T] += 1
    return len(Tt.keys())

In [3]:
instr0 = parse09("data/test09_1.txt")
instr = parse09("data/input09.txt")
print("Test 1:",part1(instr0))
print("Part 1:",part1(instr))

Test 1: 13
Part 1: 6464


In [4]:
import matplotlib.pyplot as plt
        
def part2(instr,l=10):
    # rope element traces
    Rt = [ defaultdict(int) for _ in range(l) ]
    # rope elements last position
    rope = [ (0,0) for _ in range(l) ]
    # initial positions
    for Kt in Rt:
        Kt[(0,0)] = 1
    k=0
    for d,n in instr:
        # print(d,n)
        for _ in range(n):
            # move Head
            Hprev = rope[0]
            H = moveHead(Hprev,d)
            Rt[0][H] += 1
            rope[0] = H
            # move other rope elements
            for i in range(1,l):
                Tprev = rope[i]
                T = moveTail(H,Hprev,Tprev)
                Rt[i][T] += 1
                rope[i] = T
                H = T
                Hprev = Tprev
    return len(Rt[9].keys())

In [5]:
instr1 = parse09("data/test09_2.txt")

print("Test 2:",part2(instr1))
print("Part 2:",part2(instr))

Test 2: 36
Part 2: 2604
